In [1]:
import cv2
import numpy as np
import os
import time
from matplotlib import pyplot as plt
import time
from HandTrackerRenderer import HandTrackerRenderer
from HandTrackerEdge import HandTracker
from FPS import FPS, now
from tensorflow import keras

In [2]:
#labelling  images
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [38]:
#importing variables
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Collected_Data') 
# Actions that we try to detect
actions = np.array(['a','am','are','do','for','have','Hello','I','is','it','pretty','sell','shirt'
                    ,'skirt','so','the','to','wear','you','one','two','three','four','five','six'
                    ,'about','bad','look','think'])

# Thirty videos worth of data
no_sequences =60

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

label_map = {label:num for num, label in enumerate(actions)}

In [39]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        res = np.load(os.path.join(DATA_PATH, action, str(action)+'_'+ str(sequence)+'.npy'))
        sequences.append(res)
        labels.append(label_map[action])

In [51]:
sign_model = keras.models.load_model("action19_LSTM_New.h5")

In [52]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [67]:
def extractData(hands) :
    rh_data = np.zeros((21, 3))
    lh_data = np.zeros((21, 3))
    for hand in hands : 
        Label = hand.label
        if Label == 'left' : 
            lh_data = hand.norm_landmarks 
        elif Label == 'right' : 
            rh_data = hand.norm_landmarks
    return lh_data,rh_data

def most_freq(List) :
    return max(set(List), key = List.count)


In [78]:
tracker = HandTracker(solo = False)

renderer = HandTrackerRenderer(
        tracker=tracker,
        output=None)

Palm detection blob     : C:\Secret\College\Senior\SIgnExactEnglish\models\palm_detection_sh4.blob
Landmark blob           : C:\Secret\College\Senior\SIgnExactEnglish\models\hand_landmark_lite_sh4.blob
PD post processing blob : C:\Secret\College\Senior\SIgnExactEnglish\custom_models\PDPostProcessing_top2_sh1.blob
Sensor resolution: (1920, 1080)
Internal camera FPS set to: 36
Internal camera image size: 1152 x 648 - pad_h: 252
Creating pipeline...
Creating Color Camera...
Creating Palm Detection pre processing image manip...
Creating Palm Detection Neural Network...
Creating Palm Detection post processing Neural Network...
Creating Hand Landmark pre processing image manip...
Creating Hand Landmark Neural Network (2 threads)...
Pipeline created.
Pipeline started - USB speed: SUPER


In [108]:
# 1. New detection variables
lh = []
rh = []
sentence = []
pred = []
threshold = 0.75
word_threshold = 25
now_sign=[]
font=cv2.FONT_HERSHEY_SIMPLEX
color=(0,255,0)
size = 1
thickness=2
while True:
    frame, hands, bag = tracker.next_frame()
    if frame is None: break
    # Draw hands
    hand = renderer.draw(frame, hands, bag)
    key = renderer.waitKey(delay=1)
    lh_data,rh_data = extractData(hands)
    lh.append(lh_data)
    rh.append(rh_data)
    lh_used=lh[-30:]
    rh_used=rh[-30:]
    lrh = np.concatenate([lh_used,rh_used])
    lrh_used = lrh.reshape(lrh.shape[0],-1)
    cv2.putText(frame,f"data = {len(lrh)}",(20,50),font,size,color,thickness)
    # predictions logic
    if len(lrh) == 60:
        res = sign_model.predict(np.expand_dims(lrh_used, axis=0))[0]
        predicted_sign = actions[np.argmax(res)]
        confidence = res[np.argmax(res)]
        if (confidence > threshold) :
            pred.append(predicted_sign)
        if len(pred) >= 30 : 
            if (most_freq(pred[-30:]) not in now_sign) & (pred.count(most_freq(pred[-30:])) >= word_threshold) :
                target = most_freq(pred[-30:])
                now_sign = [target]
                print(target)
            #print(predicted_sign)
            #print(confidence)
            #now_sign=[actions[np.argmax(res)]]
        #lh = []
        #rh = []
        lh.pop(0)
        rh.pop(0)
        print(pred)
    if key == 27 or key == ord('q'):
        break
renderer.exit()

In [77]:
tracker.exit()